In [ ]:
#from tqdm import tqdm_notebook
from tqdm.notebook import tqdm, tnrange
import time

In [ ]:
from osgeo import gdal
import numpy as np
import glob
import sys, os
sys.path.insert(0, os.path.dirname(os.getcwd()))
from utils import gdal_processing as gp
import seaborn as sns
import pandas as pd
from skimage.measure import block_reduce
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
from utils.data_reader import interpPatches

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

In [ ]:
save_path_figs = '/scratch2/Dropbox/Dropbox/Apps/Overleaf/activelearning_remotesensing/figures/'

# 1. Sumatra and Peninsular Malaysia

#### Reading GT

In [ ]:
# path_gt = '/scratch/andresro/leon_work/barry_palm/data/labels/manual_annotations'
path_gt = '/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/manual_annotations'

tiles_gt_all = glob.glob(path_gt+'/T46*/palm*')
tiles_gt_all += glob.glob(path_gt+'/T47*/palm*')
tiles_gt_all += glob.glob(path_gt+'/T48*/palm*')

tiles_gt_all = sorted(tiles_gt_all)
print(len(tiles_gt_all))

In [ ]:
rois_gt_all = np.array([gp.get_positive_area_folder(tile_) for tile_ in tiles_gt_all])

def middle_roi(roi_list):
    return np.array([roi_list[:,0]+roi_list[:,2],roi_list[:,1]+roi_list[:,3]]).T/2

rois_middle = middle_roi(rois_gt_all)



In [ ]:
gdf = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(rois_middle[:,0], rois_middle[:,1]))


In [ ]:
# path_ = '/scratch/andresro/leon_work/barry_palm/data/labels/manual_annotations'
# index_  = [tiles_available.index(path_+x) for x in out_dict['10a']]
# index_bool = [x in index_ for x in list(gdf.index)]

In [ ]:
#gdf['Sample'] = 'No'
#gdf['Sample'].loc[nonuniform_samples > 0] = 'Non Uniform'
#gdf['Sample'].loc[uniform_samples > 0] = 'Random'

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
countries = geopandas.read_file("/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/countries/3countries.shp")


fig, ax = plt.subplots(figsize=(5,5))
world.plot(ax=ax,
    color='lightgray', edgecolor='white')

countries.plot(ax=ax,
    color='lightgray', edgecolor='white')


gdf.plot(ax=ax, marker='.')

minx, miny, maxx, maxy = gdf.total_bounds
margin_x = 2
margin_y = 2

ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)
plt.title('Available samples')
# plt.savefig('active_learning_samples_10a.png')


#### Choosing Base train and val samples

In [ ]:
N = len(tiles_gt_all)
np.random.seed(1)
index = np.arange(N)
np.random.shuffle(index)

In [ ]:
index[0:10]

In [ ]:
index_train = index[0:10]
index_val = index[10:20]
index_avail = index[20:]

In [ ]:
tiles_train = np.array(tiles_gt_all)[index_train]
tiles_val = np.array(tiles_gt_all)[index_val]
tiles_avail = np.array(tiles_gt_all)[index_avail]

assert len(set(tiles_train).intersection(set(tiles_val))) == 0

assert len(set(tiles_val).intersection(set(tiles_avail))) == 0
assert len(set(tiles_train).intersection(set(tiles_avail))) == 0



In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
countries = geopandas.read_file("/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/countries/3countries.shp")


#fig, ax = plt.subplots(figsize=(5,5))
plt.figure(figsize=(6,6))
ax = plt.axes(projection=ccrs.PlateCarree())


world.plot(ax=ax,
    color='lightgray', edgecolor='white')

countries.plot(ax=ax,
     color='lightgray', edgecolor='white')



gdf.plot(ax=ax,legend=True, marker='.',
          edgecolor='black',
          facecolor='none',
         # color='green',
         label='Available'
        )

gdf.loc[index_train].plot(ax=ax, marker='.',
                               color='orange',
               label='Train'
        )

# gdf_train.plot(ax=ax, marker='.',
#                                #  color='red',
#                label='Train'
#         )
gdf.loc[index_val].plot(ax=ax, marker='.',
color='blue',
                label='Val'
         )




minx, miny, maxx, maxy = gdf.total_bounds
margin_x = 2
margin_y = 2



ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)



gl = ax.gridlines(
   crs=ccrs.PlateCarree(),
    draw_labels=True) #x_inline=False, y_inline=False)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'rotation':'vertical'}



ax.legend(loc='lower left')
# lgd = ax.legend(bbox_to_anchor=(1, 0.75), loc='upper left', ncol=1,
#          title="Sample Type")
# plt.title('Fully labelled example \n in Sumatra and Peninsular Malaysia \n\n\n',)
plt.savefig(save_path_figs+'fullylabelled_example.png', bbox_inches='tight',dpi=300)


In [ ]:
[x.split('manual_annotations')[-1] for x in tiles_train]

In [ ]:
out_dict = {'train':
           [x.split('manual_annotations')[-1] for x in tiles_train],
           'val':
           [x.split('manual_annotations')[-1] for x in tiles_val]}

In [ ]:
import json
#save_path = '/home/pf/pfstaff/projects/andresro/barry_palm/data/2A/datasets'
#filename = save_path+'/palm4748a_base.json'
#with open(filename, 'w') as fp:
#    json.dump(out_dict, fp)

In [ ]:
import json
save_path = '/home/pf/pfstaff/projects/andresro/barry_palm/data/2A/datasets'
filename = save_path+'/palm4748a_base.json'
with open(filename, 'rb') as fp:
   out_dict = json.load(fp)

In [ ]:
val_tiles = {x.split('/')[1] for x in out_dict['val']}
print(len(val_tiles))
' '.join(val_tiles)

In [ ]:
opt_50 = ['/T47MQU/palm_group1_Bischel',
 '/T47NMD/palm_group3_Bischel',
 '/T47NQA/palm_group2',
 '/T47NNC/palm_group1_Brunner',
 '/T48MUC/palm_group1_Bischel',
 '/T47NPC/palm_group1_jacopo',
 '/T47NLE/palm_group3_hanlon',
 '/T48MUB/palm_group3_Brunner',
 '/T47NRA/palm_group3_Bischel',
 '/T48NUG/palm_group1',
 '/T47NRA/palm_group1_Bischel',
 '/T48MTC/palm_group1_Bischel',
 '/T47MQV/palm_group2',
 '/T47NPC/palm_group2_jacopo',
 '/T48MUA/palm_group1_Bischel',
 '/T48NTK/palm_group2',
 '/T47NNC/palm_group3_Bischel',
 '/T48MTE/palm_group1',
 '/T47NPA/palm_group1_Bischel',
 '/T47NMC/palm_group2_Bischel',
 '/T47NQB/palm_group1',
 '/T47MRV/palm_group1_siri',
 '/T47NMD/palm_group1_Bischel',
 '/T47NKF/palm_group1_bischel',
 '/T48MVB/palm_group3_Bischel',
 '/T48MWB/palm_group3_Bischel',
 '/T47NLD/palm_group1_florian',
 '/T48MUB/palm_group2_Brunner',
 '/T47NPC/palm_group3_jacopo',
 '/T48MUA/palm_group2_Bischel',
 '/T47NNB/palm_group3_stephane',
 '/T47NMC/palm_group1_Bischel',
 '/T47NLB/palm_group1_florian',
 '/T48MTC/palm_group3_Bischel',
 '/T47NLE/palm_group1_hanlon',
 '/T47NQD/palm_group1',
 '/T47NRF/palm_group1',
 '/T47NPA/palm_group3_Bischel',
 '/T48MUB/palm_group1_Brunner',
 '/T48MUB/palm_group3_Bischel',
 '/T47NRG/palm_group3',
 '/T47MQV/palm_group1',
 '/T47MQU/palm_group3_Bischel',
 '/T48MVB/palm_group1_Bischel',
 '/T48NUH/palm_group2',
 '/T47NRG/palm_group1',
 '/T47NKF/palm_group2_bischel',
 '/T47NMD/palm_group2_Bischel',
 '/T48MVV/palm_group3_matthew',
 '/T46NGL/palm_group2_janmathias']

In [ ]:
opt_50t = ['/T46NHL/palm_group4_bischel',
 '/T47MMU/palm_group1_julia',
 '/T47MMU/palm_group2_julia',
 '/T47MMU/palm_group3_julia',
 '/T47MQU/palm_group1_Bischel',
 '/T47MQV/palm_group2',
 '/T47MRS/palm_group2_Bischel',
 '/T47MRV/palm_group1_siri',
 '/T47NKF/palm_group1_bischel',
 '/T47NLB/palm_group2_florian',
 '/T47NLD/palm_group1_florian',
 '/T47NLE/palm_group3_hanlon',
 '/T47NMC/palm_group2_Bischel',
 '/T47NMD/palm_group1_Bischel',
 '/T47NMD/palm_group3_Bischel',
 '/T47NNC/palm_group1_Brunner',
 '/T47NNC/palm_group3_Bischel',
 '/T47NND/palm_group1_Brunner',
 '/T47NPA/palm_group1_Bischel',
 '/T47NPA/palm_group2_Bischel',
 '/T47NPB/palm_group1',
 '/T47NPC/palm_group1_jacopo',
 '/T47NPC/palm_group2_jacopo',
 '/T47NQA/palm_group2',
 '/T47NQB/palm_group1',
 '/T47NQD/palm_group1',
 '/T47NQD/palm_group2',
 '/T47NQE/palm_group1',
 '/T47NRA/palm_group1_Bischel',
 '/T47NRA/palm_group3_Bischel',
 '/T47NRF/palm_group1',
 '/T47NRF/palm_group3',
 '/T48MTC/palm_group1_Bischel',
 '/T48MTE/palm_group1',
 '/T48MUA/palm_group1_Bischel',
 '/T48MUB/palm_group3_Brunner',
 '/T48MUC/palm_group1_Bischel',
 '/T48MVB/palm_group3_Bischel',
 '/T48MVV/palm_group1_matthew',
 '/T48MVV/palm_group2_matthew',
 '/T48MWB/palm_group1_Bischel',
 '/T48MWB/palm_group2_Bischel',
 '/T48MWB/palm_group3_Bischel',
 '/T48MWV/palm_group1_Bischel',
 '/T48MWV/palm_group2_Bischel',
 '/T48MWV/palm_group3_Bischel',
 '/T48NTH/palm_group2',
 '/T48NTK/palm_group2',
 '/T48NUG/palm_group1',
 '/T48NUH/palm_group2']

In [ ]:
set(out_dict['train']).intersection(set(opt_50))

In [ ]:
len(set(opt_50t).intersection(set(opt_50)))

In [ ]:
# Filter out val areas

#tilenames_val = ['T47NQD', 'T48NUH'] # areas in peninsula
#tilenames_val += ['T47NKF', 'T47MPV', 'T48MVB'] # areas in Sumatra

#tilenames_tr = ['T47NRG','T47NPF','T47NRF','T47NQE','T47NRE']
#tilenames_tr += ['T48NTK','T48NTJ','T48NTH','T48NUG']   


#def isin(string_,list_):
#    return np.any([x in string_ for x in list_])

#tiles_train = [x for x in tiles_gt_all if isin(x,tilenames_tr)]
#tiles_val = [x for x in tiles_gt_all if isin(x,tilenames_val)]

#tiles_available = [x for x in tiles_gt_all if not isin(x,tilenames_val) and not isin(x,tilenames_tr)]

#print(f'all {len(tiles_gt_all)}, train {len(tiles_train)}, val {len(tiles_val)}, remaining {len(tiles_available)}')

#rois_available = np.array([gp.get_positive_area_folder(tile_) for tile_ in tiles_available])
#rois_train = np.array([gp.get_positive_area_folder(tile_) for tile_ in tiles_train])
#rois_val = np.array([gp.get_positive_area_folder(tile_) for tile_ in tiles_val])


In [ ]:
#def middle_roi(roi_list):
#    return np.array([roi_list[:,0]+roi_list[:,2],roi_list[:,1]+roi_list[:,3]]).T/2

#rois_middle = middle_roi(rois_available)
#rois_middle_train = middle_roi(rois_train)
#rois_middle_val = middle_roi(rois_val)

#rois_middle_train = np.array([rois_train[:,0]+rois_train[:,2],rois_train[:,1]+rois_train[:,3]]).T/2

### Reading ensembe predictions

In [ ]:
path = '/scratch/andresro/leon_work/sparse/inference/palm4748a_simpleA9_soft_ens5'

#path = '/scratch/andresro/leon_work/sparse/inference/palm4650_simpleA9_soft_mc5'

In [ ]:
tilenames = list({os.path.basename(x).split('_')[1] for x in glob.glob(path+'/R*')})
len(tilenames)

For every sample $i$ and dimension $j$:

$$ \sum_i ||X_i - Y ||^2_2 = \sum_i \big( \sum_j X_{i,j}^2 - 2 \sum_j (X_{i,j}*Y_j) +  \sum_j Y_j^2 \big)  $$

$$ \sum_j \big( \sum_i X_{i}^2 - 2 Y_j\sum_i X_{i} + I* Y_j^2 \big)  $$

In [ ]:
# Aggregate same locations
for tilename in tilenames:
    fname = f'{path}/{tilename}-5_preds_last.npz'
    if not os.path.isfile(fname):
        pred_dirs = glob.glob(f'{path}/*{tilename}/*.npz')

        # sum over several days in same locations
        x_sum1 = []
        x2_sum1 = []
        n1 = []

        for dir_ in pred_dirs:
            preds_last = np.load(dir_)

            x_sum2, x2_sum2, n2 = preds_last['x_sum'], preds_last['x2_sum'], preds_last['n']
            x_sum1.append(x_sum2)
            x2_sum1.append(x2_sum2)
            n1.append(n2)

        x_sum1 = np.stack(x_sum1,axis=0).sum(axis=0)
        x2_sum1 = np.stack(x2_sum1,axis=0).sum(axis=0)
        n1 = np.stack(n1,axis=0).sum(axis=0)

        np.savez(fname,x_sum=x_sum1,x2_sum=x2_sum1,
                        n=n1, lonlat=preds_last['lonlat'])
        print(fname,'saved')


In [ ]:
compression_ = '0'

In [ ]:
x_sum = []
x2_sum = []
n = []
lonlat = []
std_patches=[]

is_reload = False
fname_all = f'{path}/all_statistics.npz'

if not os.path.isfile(fname_all) or is_reload:
    print('re-processing and saving')
    
    for tilename in tqdm(tilenames):

        fname = f'{path}/{tilename}-5_preds_last.npz'
        if os.path.isfile(fname):
            preds_last = np.load(fname)
            x_sum0, x2_sum0, n0, lonlat0 = preds_last['x_sum'], preds_last['x2_sum'], preds_last['n'], preds_last['lonlat']
            
            
            # Adding nan values to patch statistics
            down100_file = f'{path}/{tilename}_5_preds_reg_{compression_}_{compression_}_down100.vrt'
            ds = gdal.Open(down100_file)
            down_array = ds.ReadAsArray()
            is_nan_array = interpPatches((down_array==99), ref_shape=(100,100)).squeeze() > 0.5
            is_nan_array = is_nan_array.reshape(-1)

            x_sum0[is_nan_array] = np.nan
            x2_sum0[is_nan_array] = np.nan
            n0[is_nan_array] = 0
            
            x_sum.append(x_sum0)
            x2_sum.append(x2_sum0)
            n.append(n0)
            lonlat.append(lonlat0)

            # Adding mean uncertainty per patch
            std_file = f'{path}/std/{tilename}_5_preds_reg_{compression_}_std_{compression_}.tif'
            ds = gdal.Open(std_file)
            std_array = ds.ReadAsArray()
            std_array[std_array == 99] = np.nan
            std_patch = block_reduce(std_array,(110,110),np.mean)
            std_patches.append(std_patch.reshape(-1))

    x_sum = np.stack(x_sum,axis=0)
    x2_sum = np.stack(x2_sum,axis=0)
    n = np.stack(n,axis=0)
    lonlat = np.stack(lonlat,axis=0)
    std_patches = np.stack(std_patches,axis=0)

    np.savez(fname_all,x_sum=x_sum,
            x2_sum=x2_sum,
            n=n, lonlat=lonlat,
            std_patches = std_patches)    
else:
    print('reading',fname_all)
    preds_all = np.load(fname_all)

    x_sum = preds_all['x_sum']
    x2_sum = preds_all['x2_sum']
    n = preds_all['n']
    lonlat = preds_all['lonlat']
    std_patches = preds_all['std_patches']

In [ ]:
#is_nan_patch = np.isnan(x_sum[...,0])
is_nan_patch = n == 0
#is_nan_patch = np.logical_or(n == 0,np.isnan(std_patches))

In [ ]:
# is_nan_patch = np.zeros_like(n) != 0

In [ ]:
is_nan_patch.mean()

In [ ]:
is_nan_patch.shape,std_patches.shape

In [ ]:
plt.scatter(lonlat[is_nan_patch][...,0], lonlat[is_nan_patch][...,1],
            #c=np.log(weight_sorted[:top_k]),
            # c=weight_sorted[:top_k],
            cmap='magma_r', marker='.')

In [ ]:
plt.scatter(lonlat[~is_nan_patch][...,0], lonlat[~is_nan_patch][...,1],
            #c=np.log(weight_sorted[:top_k]),
            # c=weight_sorted[:top_k],
            cmap='magma_r', marker='.')

In [ ]:
x_sum[~is_nan_patch].shape

In [ ]:
mean_global = np.sum(x_sum[~is_nan_patch], axis=0) / n.sum()

# error means quantization error not std. dev.
error_patch = (x2_sum[~is_nan_patch] - 2*x_sum[~is_nan_patch]*mean_global + n.sum() * (mean_global**2))

# error_patch = np.clip(error_patch,0,np.percentile(error_patch,99))

error_global = np.sum(error_patch, axis=0)

In [ ]:
mean_global.shape, error_patch.shape

In [ ]:
p_min = 0
p_max = 99

uncertainty = np.nan_to_num(std_patches[~is_nan_patch][...,np.newaxis])

weight_div = error_patch.sum(axis=-1).reshape(-1) / np.sum(error_patch)

weight_div = np.clip(weight_div,np.percentile(weight_div,p_min),np.percentile(weight_div,p_max))
weight_div = weight_div / weight_div.sum()

weight_unc = (uncertainty).sum(axis=-1).reshape(-1) / np.sum(uncertainty)

#weight_unc = np.clip(weight_unc,np.percentile(weight_unc,p_min),np.percentile(weight_unc,p_max))
#weight_unc = weight_unc / weight_unc.sum()


weight_score = (weight_div + weight_unc)/2.0
weight_score = weight_score/weight_score.sum()

In [ ]:
max_perc = 99

f, ax = plt.subplots(2,2, figsize=(10,5))
ax[0,0].plot(weight_div,'x')
ax[0,1].plot(weight_unc,'x')

_ = ax[1,0].hist(np.clip(weight_div,0,np.percentile(weight_div,max_perc)))
_ = ax[1,1].hist(np.clip(weight_unc,0,np.percentile(weight_unc,max_perc)))

In [ ]:
np.percentile(weight_div,90), np.percentile(weight_div,99), np.percentile(weight_div,100)

In [ ]:
#index = np.random.choice(weight.shape[0],100000)
p_min = 0
p_max = 100

index_inlier = np.logical_and.reduce(
                      (weight_div > np.percentile(weight_div,p_min),
                       weight_div < np.percentile(weight_div,p_max),
                       weight_unc > np.percentile(weight_unc,p_min),
                       weight_unc < np.percentile(weight_unc,p_max),
                      ))


g = sns.jointplot(x=weight_div[index_inlier], y=weight_unc[index_inlier], kind="hex",
                #  xlim=(np.percentile(weight,p_min),np.percentile(weight,p_max)),
                # ylim=(np.percentile(weight1,p_min),np.percentile(weight1,100))) #, data=tips))
                 )

g.ax_joint.set_xlabel('Quant Error')
g.ax_joint.set_ylabel('Uncertainty')
#g.ax_joint.set_xlim(0,1)


# g.ax_joint.set_xlim(np.percentile(weight_div,p_min),np.percentile(weight_div,80))
#g.ax_joint.set_ylim(np.percentile(weight1,p_min),np.percentile(weight1,100))
# g.ax_joint.set_xticks([])
# g.ax_joint.set_yticks([])



# plt.savefig(save_path_figs + 'uncertainty_vs_quant_palm4748.png', bbox_inches='tight', dpi= 300)

In [ ]:
ids_sorted = np.argsort(weight_score)[::-1] 

weight_sorted = weight_score[ids_sorted]
lonlat_sorted = lonlat[~is_nan_patch][ids_sorted]

In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

In [ ]:
# index_train = [x.split('manual_annotations')[-1] in out_dict['train'] for x in tiles_gt_all]
# index_val = [x.split('manual_annotations')[-1] in out_dict['val'] for x in tiles_gt_all]

# index_bool = [x.split('manual_annotations')[-1] in out_dict['30opt'] for x in tiles_gt_all]


In [ ]:
len(index_train)

In [ ]:
lonlat_sorted[0]

In [ ]:
rois_middle[0]

In [ ]:
rois_middle.shape

In [ ]:
N_active = 30

gdf = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(rois_middle[:,0], rois_middle[:,1]))

gdf = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(rois_middle[:,0], rois_middle[:,1]))


gdf_active = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(lonlat_sorted[:N_active,0], lonlat_sorted[:N_active,1]))


In [ ]:
top_k = 1000
plt.scatter(lonlat_sorted[:top_k,0], lonlat_sorted[:top_k,1],
            #c=np.log(weight_sorted[:top_k]),
            c=weight_sorted[:top_k],
            cmap='magma_r', marker='.')

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
countries = geopandas.read_file("/scratch/andresro/leon_work/barry_palm/data/labels/countries/3countries.shp")


plt.figure(figsize=(6,6))
ax = plt.axes(projection=ccrs.PlateCarree())


world.plot(ax=ax,
    color='lightgray', edgecolor='white')

#countries.plot(ax=ax,
#    color='lightgray', edgecolor='white')

gdf.loc[index_train].plot(ax=ax, marker='.',
                               color='orange',
               label='Train'
        )

gdf.loc[index_val].plot(ax=ax, marker='.',
color='blue',
                label='Val'
         )

gdf_active.plot(ax=ax, marker='.',
                                 color='red', 
                                 label='Active'
        )


minx, miny, maxx, maxy = gdf.total_bounds
margin_x = 2
margin_y = 2


minx1, miny1, maxx1, maxy1 = gdf_active.total_bounds

minx = min(minx,minx1)
miny = min(miny,miny1)
maxx = max(maxx,maxx1)
maxy = max(maxy,maxy1)


ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)




gl = ax.gridlines(
   crs=ccrs.PlateCarree(),
    draw_labels=True) #x_inline=False, y_inline=False)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'rotation':'vertical'}


# ax.legend(bbox_to_anchor=(1, 0.75), loc='upper left', ncol=1,
#          title="Sample Type")
#ax.legend() # loc='lower left')

# plt.title('Active Learning Selected Samples\n\n')
# plt.savefig(save_path_figs + 'active_learning_samples_10opt.', bbox_inches='tight', dpi= 300)


In [ ]:
import simplekml

kml = simplekml.Kml()
for key, val in enumerate(lonlat_sorted[:N_active]):
    #lat, lon = gp.to_latlon(val[1],val[0],ds_pred)
    lon, lat = val
    pnt = kml.newpoint(name=str(key), coords=[(lon, lat)])
    
# kml.save(f'/scratch2/Dropbox/Dropbox/0_phd/temp/{N_active}_New_samples_palm4.kml')

#### looking for the closest patch for each labeled area

In [ ]:
def closest_node(node, nodes):
    dist_2 = np.sum((nodes - node)**2, axis=-1)
    min_index = np.argmin(dist_2)
    dist_min = dist_2[min_index]
    return min_index, dist_min
#     return np.divmod(min_index,dist_2.shape[-1]),dist_min

In [ ]:
patch_id_avail = []
distances = []
for roi in rois_middle[index_avail]:

    node_id, distance = closest_node(roi,lonlat[~is_nan_patch])
    patch_id_avail.append(node_id)
    distances.append(distance)

In [ ]:
plt.scatter(lonlat[~is_nan_patch][patch_id_avail][:,0], lonlat[~is_nan_patch][patch_id_avail][:,1],
# plt.scatter(lonlat[~is_nan_patch][core_set][patch_id_gt][:,0], lonlat[~is_nan_patch][core_set][patch_id_gt][:,1],            
# plt.scatter(lonlat.reshape(-1,2)[patch_id_gt][:,0], lonlat.reshape(-1,2)[patch_id_gt][:,1],
            #c=np.log(weight_sorted[:top_k]),
#             c=weight_sorted[:top_k],
#             cmap='magma_r',
            marker='o')
plt.scatter(rois_middle[:,0], rois_middle[:,1],
            #c=np.log(weight_sorted[:top_k]),
#             c=weight_sorted[:top_k],
#             cmap='magma_r',
            marker='x')

#### Using top-k from each weight independently

In [ ]:
weight_unc[patch_id_avail].shape,index_avail.shape

In [ ]:

C=50


core_set = set()

unc_sorted = np.argsort(weight_unc[patch_id_avail])[::-1]
div_sorted = np.argsort(weight_div[patch_id_avail])[::-1]

i = 0
j = 0
while len(core_set) < C:
    
    if np.mod(i+j,2) == 0:
        core_set.add(unc_sorted[i])
        i+=1
    else:
        core_set.add(div_sorted[j])
        j+=1
        
core_set = np.array(list(core_set))

In [ ]:
core_set_score = np.argsort(weight_score[patch_id_avail])[::-1][:C]

In [ ]:
len(set(core_set).intersection(set(core_set_score)))

In [ ]:
g = sns.jointplot(x=weight_div, y=weight_unc, kind="hex")

g.ax_joint.set_xlabel('Quant Error')
g.ax_joint.set_ylabel('Uncertainty')
# g.ax_joint.set_xlim(np.percentile(weight_div,p_min),np.percentile(weight_div,80))
#g.ax_joint.set_ylim(np.percentile(weight1,p_min),np.percentile(weight1,100))
# g.ax_joint.set_xticks([])
# g.ax_joint.set_yticks([])
g.ax_joint.scatter(x=weight_div[patch_id_avail],
                   y=weight_unc[patch_id_avail], c='green')
#g.ax_joint.scatter(x=weight_div[patch_id_avail][core_set],
#                   y=weight_unc[patch_id_avail][core_set], c='white', marker='+')
g.ax_joint.scatter(x=weight_div[patch_id_avail][core_set_score],
                   y=weight_unc[patch_id_avail][core_set_score],
                   marker='o',
                   edgecolor='red',
                   facecolor='none',
                   )

# plt.savefig(save_path_figs + 'uncertainty_vs_quant_all.png', bbox_inches='tight', dpi= 300)

In [ ]:
g = sns.jointplot(x=weight_div, y=weight_unc, kind="hex")

g.ax_joint.set_xlabel('Quant Error')
g.ax_joint.set_ylabel('Uncertainty')
# g.ax_joint.set_xlim(np.percentile(weight_div,p_min),np.percentile(weight_div,80))
#g.ax_joint.set_ylim(np.percentile(weight1,p_min),np.percentile(weight1,100))
# g.ax_joint.set_xticks([])
# g.ax_joint.set_yticks([])
g.ax_joint.scatter(x=weight_div[patch_id_avail], y=weight_unc[patch_id_avail], c='green')
g.ax_joint.scatter(x=weight_div[patch_id_avail][core_set], y=weight_unc[patch_id_avail][core_set], c='red', marker='x')

# plt.savefig(save_path_figs + 'uncertainty_vs_quant_all.png', bbox_inches='tight', dpi= 300)

In [ ]:
out_dict = {}
M_list = [5, 10, 15, 30, 50]

for M in M_list:
    core_set = set()

    unc_sorted = np.argsort(weight_unc[patch_id_avail])[::-1]
    div_sorted = np.argsort(weight_div[patch_id_avail])[::-1]

    i = 0
    j = 0
    while len(core_set) < M:

        if np.mod(i+j,2) == 0:
            core_set.add(unc_sorted[i])
            i+=1
        else:
            core_set.add(div_sorted[j])
            j+=1

    core_set = np.array(list(core_set))
    
    selected_files = sorted(tiles_avail[core_set])
    key_ = f'{M}optt'
    out_dict[key_] = [x.split('manual_annotations')[-1] for x in selected_files]


In [ ]:
out_dict.keys()

In [ ]:
import json
save_path = '/home/pf/pfstaff/projects/andresro/barry_palm/data/2A/datasets'
filename = save_path+'/palm4748a_activesamples_topk.json'
with open(filename, 'w') as fp:
    json.dump(out_dict, fp)

#with open(filename, 'r') as fp:
#    out_dict = json.load(fp)


## Using core-set for a k-mean solution

#### Sample C samples to create a core-set

In [ ]:
index_avail

In [ ]:
C = 10000
N_samples = weight_score.shape[0]

# w_ = weight_score[n[~is_nan_patch] > 0]/weight_score[n[~is_nan_patch] > 0].sum()
# w_ = weight_score[n.reshape(-1) > 0]/weight_score[n.reshape(-1) > 0].sum()
# w_ = weight_score[np.logical_and(n.reshape(-1) > 0,np.percentile(weight_score,99))]/weight_score[np.logical_and(n.reshape(-1) > 0,np.percentile(weight_score,99))].sum()
# w_ = weight_unc[n.reshape(-1) > 0]/weight_unc[n.reshape(-1) > 0].sum()

samples_id = np.arange(N_samples) # [n[~is_nan_patch] > 0]

np.random.seed(1)

core_set = np.random.choice(samples_id,size=C, replace=True, p=weight_score)

# adding the labeled samples to the coreset
core_set = np.concatenate((core_set,patch_id_gt))

# ids_sorted = np.argsort(weight_score)[::-1] 

# core_set = samples_id[ids_sorted][:C]

core_set_weight = 1 / (C* weight_score[core_set])
# core_set_weight = 1 / (C * weight_unc[core_set])

# key_ = f'{M}{rand_option}'

In [ ]:
g = sns.jointplot(x=weight_div[core_set], y=weight_unc[core_set], kind="hex")

g.ax_joint.set_xlabel('Quant Error')
g.ax_joint.set_ylabel('Uncertainty')
# g.ax_joint.set_xlim(np.percentile(weight_div,p_min),np.percentile(weight_div,80))
#g.ax_joint.set_ylim(np.percentile(weight1,p_min),np.percentile(weight1,100))
# g.ax_joint.set_xticks([])
# g.ax_joint.set_yticks([])
g.ax_joint.scatter(x=weight_div[patch_id_gt], y=weight_unc[patch_id_gt], c='red')

# plt.savefig(save_path_figs + 'uncertainty_vs_quant_all.png', bbox_inches='tight', dpi= 300)

In [ ]:
x_mean_coreset = x_sum[~is_nan_patch][core_set] / (n[~is_nan_patch][core_set][...,np.newaxis])

In [ ]:
# lonlat.reshape(-1,2)

In [ ]:
# patch_id_gt = []
# distances = []
# for roi in rois_middle[index_avail]:

#     node_id, distance = closest_node(roi,lonlat[~is_nan_patch][core_set])
#     patch_id_gt.append(node_id)
#     distances.append(distance)


In [ ]:
from sklearn.manifold import TSNE

Y = TSNE(n_components=2).fit_transform(x_mean_coreset)

In [ ]:
g = sns.jointplot(Y[:,0],Y[:,1],kind='kde')

# sns.jointplot(Y[patch_id_gt][:,0],Y[patch_id_gt][:,1])
g.ax_joint.scatter(Y[-len(patch_id_gt):][:,0],Y[-len(patch_id_gt):][:,1], c='red')
# g.ax_joint.scatter(Y[patch_id_gt][center_pixels_index][:,0],Y[patch_id_gt][center_pixels_index][:,1], c='white', marker='.', facecolor=None)

In [ ]:
# error_patch1 = np.array([error_patch[id_] for id_ in patch_id_gt])
# std_patch1 = np.array([std_patches[id_] for id_ in patch_id_gt])

x_mean_labeled = x_sum[~is_nan_patch][patch_id_gt]/ (n[~is_nan_patch][patch_id_gt][...,np.newaxis])



In [ ]:
# uncertainty1 = np.nan_to_num(std_patch1[...,np.newaxis])
# weight = error_patch1.sum(axis=-1).reshape(-1) / error_patch1.sum()
# weight1 = (uncertainty1).sum(axis=-1).reshape(-1) / (uncertainty1).sum()

# weight2 = (weight + weight1)/2.0


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
n_clusters = 30

model = KMeans(n_clusters=n_clusters, random_state=0)
model.fit(x_mean_coreset[-len(patch_id_gt):], sample_weight=core_set_weight[-len(patch_id_gt):])

In [ ]:
cluster_id = model.predict(x_mean_coreset)
center_pixels_index_coreset = np.argmin(model.transform(x_mean_coreset), axis=0)


center_pixels_index = np.argmin(model.transform(x_mean_labeled), axis=0)

In [ ]:
id_min_weight = np.argsort(weight_score[patch_id_gt])[::-1][:n_clusters]

In [ ]:
plt.scatter(Y[:,0],Y[:,1], c=cluster_id,cmap='tab20')


plt.scatter(Y[center_pixels_index_coreset][:,0],Y[center_pixels_index_coreset][:,1],color='red')

In [ ]:
g = sns.jointplot(Y[:,0],Y[:,1],kind='kde')

# sns.jointplot(Y[patch_id_gt][:,0],Y[patch_id_gt][:,1])
g.ax_joint.scatter(Y[-len(patch_id_gt):][:,0],Y[-len(patch_id_gt):][:,1], c='red')
g.ax_joint.scatter(Y[-len(patch_id_gt):][center_pixels_index][:,0],Y[-len(patch_id_gt):][center_pixels_index][:,1],  c='white', marker='.', facecolor=None)

g.ax_joint.scatter(Y[-len(patch_id_gt):][id_min_weight][:,0],Y[-len(patch_id_gt):][id_min_weight][:,1],  c='green', marker='.', facecolor=None)
# g.ax_joint.scatter(Y[patch_id_gt][center_pixels_index][:,0],Y[patch_id_gt][center_pixels_index][:,1], c='white', marker='.', facecolor=None)
# g.ax_joint.scatter(Y[center_pixels_index_coreset][:,0],Y[center_pixels_index_coreset][:,1], c='green', marker='x')

In [ ]:
len(set(center_pixels_index)), center_pixels_index.shape

In [ ]:
# cluster_id = model.predict(x_mean_coreset)



In [ ]:
g = sns.jointplot(x=weight_div[core_set], y=weight_unc[core_set], kind="hex")


g.ax_joint.set_xlabel('Quant Error')
g.ax_joint.set_ylabel('Uncertainty')
# g.ax_joint.set_xlim(np.percentile(weight_div,p_min),np.percentile(weight_div,80))
#g.ax_joint.set_ylim(np.percentile(weight1,p_min),np.percentile(weight1,100))
# g.ax_joint.set_xticks([])
# g.ax_joint.set_yticks([])
# g.ax_joint.scatter(x=weight_div[patch_id_gt][center_pixels_index],
#                    y=weight_unc[patch_id_gt][center_pixels_index], c='red')
g.ax_joint.scatter(x=weight_div[core_set][-len(patch_id_gt):][center_pixels_index],
                   y=weight_unc[core_set][-len(patch_id_gt):][center_pixels_index],c='red')
g.ax_joint.scatter(x=weight_div[core_set][-len(patch_id_gt):][id_min_weight],
                   y=weight_unc[core_set][-len(patch_id_gt):][id_min_weight],c='green')

# plt.savefig(save_path_figs + 'uncertainty_vs_quant_coreset.png', bbox_inches='tight', dpi= 300)

In [ ]:
gdf_coreset = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(coreset_lonlat_coords[:,0], coreset_lonlat_coords[:,1]))

gdf_active = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(rois_middle[id_min_weight][:,0], rois_middle[id_min_weight][:,1]))


In [ ]:
center_pixels_index_coreset.shape

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
#countries = geopandas.read_file("/scratch/andresro/leon_work/barry_palm/data/labels/countries/3countries.shp")


plt.figure(figsize=(6,6))
ax = plt.axes(projection=ccrs.PlateCarree())


# gdf_coreset.plot(ax=ax, marker='o',
#                                  color='lightgray', 
#                                  label='Coreset'
#         )





world.plot(ax=ax,
    color='lightgray', edgecolor='white')

# countries.plot(ax=ax,
#     color='lightgray', edgecolor='white')




gdf.loc[index_train].plot(ax=ax, marker='.',
                               color='orange',
               label='Train'
        )

gdf.loc[index_val].plot(ax=ax, marker='.',
color='blue',
                label='Val'
         )


gdf.loc[index_avail].plot(ax=ax,legend=True, marker='.',
          edgecolor='black',
          facecolor='none',
         # color='green',
         label='Available'
        )


gdf_active.plot(ax=ax, marker='.',
                                 color='red', 
                                 label='Active'
        )

gdf_coreset.loc[center_pixels_index_coreset].plot(ax=ax, marker='x',
                                 color='green', 
                                 label='Coreset'
        )
minx, miny, maxx, maxy = gdf.total_bounds
margin_x = 2
margin_y = 2


minx1, miny1, maxx1, maxy1 = gdf_coreset.total_bounds

minx = min(minx,minx1)
miny = min(miny,miny1)
maxx = max(maxx,maxx1)
maxy = max(maxy,maxy1)


ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)


gl = ax.gridlines(
   crs=ccrs.PlateCarree(),
    draw_labels=True) #x_inline=False, y_inline=False)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'rotation':'vertical'}


# ax.legend(bbox_to_anchor=(1, 0.75), loc='upper left', ncol=1,
#          title="Sample Type")
ax.legend() # loc='lower left')

# plt.title('Active Learning Selected Samples\n\n')
# 

# plt.savefig(save_path_figs + 'palm4_30_ALsamples.pdf', bbox_inches='tight', dpi= 300)


In [ ]:
import simplekml

kml = simplekml.Kml()
for key, val in enumerate(center_lonlat_coords):
    #lat, lon = gp.to_latlon(val[1],val[0],ds_pred)
    lon, lat = val
    pnt = kml.newpoint(name=str(key), coords=[(lon, lat)])
    
# kml.save(f'/scratch2/Dropbox/Dropbox/0_phd/temp/{n_clusters}samples_coreset100k_combinedscore.kml')

In [ ]:
out_dict = {}
M_list = [10, 15, 30, 50, 5]

for M in M_list:
    
    model = KMeans(n_clusters=M, random_state=0)
    model.fit(x_mean_coreset, sample_weight=core_set_weight)

    cluster_id = model.predict(x_mean_coreset)

    center_pixels_index = np.argmin(model.transform(x_mean_labeled), axis=0)
    
    
    selected_files = tiles_avail[center_pixels_index]
    key_ = f'{M}optk'
    out_dict[key_] = [x.split('manual_annotations')[-1] for x in selected_files]


In [ ]:
out_dict.keys()

In [ ]:
import json
save_path = '/home/pf/pfstaff/projects/andresro/barry_palm/data/2A/datasets'
filename = save_path+'/palm4748a_activesamples_kmeans.json'
with open(filename, 'w') as fp:
    json.dump(out_dict, fp)

# with open(filename, 'r') as fp:
#    out_dict = json.load(fp)


## Random and Manual Experiments

In [ ]:
def closest_node(node, nodes):
    dist_2 = np.sum((nodes - node)**2, axis=-1)
    min_index = np.argmin(dist_2)
    dist_min = dist_2.reshape(-1)[min_index]
    return np.divmod(min_index,dist_2.shape[-1]),dist_min

In [ ]:
patch_id_gt = []
distances = []
for roi in rois_middle:

    node_id, distance = closest_node(roi,lonlat[~is_nan_patch])    
    patch_id_gt.append(node_id)
    distances.append(distance)


In [ ]:
error_patch1 = np.array([error_patch[id_] for id_ in patch_id_gt])
std_patch1 = np.array([std_patches[id_] for id_ in patch_id_gt])

In [ ]:
uncertainty1 = np.nan_to_num(std_patch1[...,np.newaxis])
weight = error_patch1.sum(axis=-1).reshape(-1) / error_patch1.sum()
weight1 = (uncertainty1).sum(axis=-1).reshape(-1) / (uncertainty1).sum()

weight2 = (weight + weight1)/2.0


In [ ]:
f, ax = plt.subplots(2,2, figsize=(10,5))
ax[0,0].plot(weight,'x')
ax[0,1].plot(weight1,'x')

max_perc = 100
_ = ax[1,0].hist(np.clip(weight,0,np.percentile(weight,max_perc)))
_ = ax[1,1].hist(np.clip(weight1,0,np.percentile(weight1,max_perc)))

In [ ]:
index_inlier

In [ ]:
import seaborn as sns

#index = np.random.choice(weight.shape[0],100000)
p_min = 0
p_max = 99


index_inlier = np.logical_and.reduce((weight >= np.percentile(weight,p_min),
                       weight <= np.percentile(weight,p_max),
                       weight1 >= np.percentile(weight1,p_min),
                     #                 weight1 > 0,
                       weight1 <= np.percentile(weight1,p_max),
                      ))


g = sns.jointplot(x=weight[index_inlier], y=weight1[index_inlier], kind="hex",
                #  xlim=(np.percentile(weight,p_min),np.percentile(weight,p_max)),
                # ylim=(np.percentile(weight1,p_min),np.percentile(weight1,100))) #, data=tips))
                 )

_ = g.ax_joint.set_xlabel('Quant Error')
_ = g.ax_joint.set_ylabel('Uncertainty')
#g.ax_joint.set_xlim(0,1)

#g.ax_joint.set_xlim(np.percentile(weight,p_min),np.percentile(weight,p_max))
#g.ax_joint.set_ylim(np.percentile(weight1,p_min),np.percentile(weight1,100))


In [ ]:
#seed_dict.keys()

In [ ]:
','.join({x.split('/')[8] for x in tiles_gt_all})

In [ ]:
import numpy as np


### Creating active learning datasets

In [ ]:
weights_sample = weight2[index_avail]

weights_sample=  weights_sample / weights_sample.sum()

In [ ]:
plt.hist(weight[index_inlier])

In [ ]:
plt.hist(weight1)

In [ ]:
plt.plot(np.sort(weights_sample))

In [ ]:
assert len(tiles_avail) ==  weights_sample.shape[0]

Choosing areas randomdummy

In [ ]:
randomdummy_groups = {}

In [ ]:
group_ = '47NRG,47NRF,47NPF,47NQE,47NRE,48NTK,47NQD,47NRD,48NTJ,47NRC,48NTH,48NUG,48NUH'.split(',')

group_ = [x.split('manual_annotations')[-1] for x in tiles_avail1 if isin(x,group_)]

randomdummy_groups['A'] = group_

#index_  = [list(tiles_gt_all).index(path_+x) for x in groupA]
print(len(group_))
#index_bool = [x in index_ for x in list(gdf.index)]


In [ ]:
group_ = '47NRG,47NRF,47NPF,47NQE,47NRE,48NTK,46NGM,46NGL,46NHL,47NKF,47NLE,47NLD,47NNE'.split(',')

group_ = [x.split('manual_annotations')[-1] for x in tiles_avail1 if isin(x,group_)]

randomdummy_groups['B'] = group_
print(len(group_))

In [ ]:
group_ = '46NGM,46NGL,46NHL,47NKF,47NLE,47NLD,47NNE,47NLD,47NMD,47NND,47NNC,47NMC'.split(',')

group_ = [x.split('manual_annotations')[-1] for x in tiles_avail1 if isin(x,group_)]
randomdummy_groups['C'] = group_
print(len(group_))

In [ ]:
group_ = '47NNC,47NMC,47NPC,47NLB,47NMB,47NNB,47NPB,47NQB,47NNA,47NPA,47NQA,47NRA'.split(',')

group_ = [x.split('manual_annotations')[-1] for x in tiles_avail1 if isin(x,group_)]
randomdummy_groups['D'] = group_
print(len(group_))

In [ ]:
group_ = '47NQA,47NRA,47MPV,47MQV,47MRV,48MTE,47MQU,47MRU,48MTD,47MRT,48MUC,47MRS,48MUB,48MVB,48MTC'.split(',')

group_ = [x.split('manual_annotations')[-1] for x in tiles_avail1 if isin(x,group_)]
randomdummy_groups['E'] = group_
print(len(group_))


In [ ]:
tiles_avail1 = np.concatenate((tiles_avail, tiles_train),axis=0)

In [ ]:
group_ = '48MUB,48MVB,48MWC,48MWB,48MUA,48MVV,48MWV,48MVU,47MRS'.split(',')

group_ = [x.split('manual_annotations')[-1] for x in tiles_avail1 if isin(x,group_)]

randomdummy_groups['F'] = group_
print(len(group_))

In [ ]:
keys_dummy = sorted(list(randomdummy_groups.keys()))

In [ ]:
order_ = np.random.choice(keys_dummy,replace=False,size=6)

In [ ]:
def create_tiles_list(dummy_order):
    tiles_ = []
    for o in dummy_order:
        tiles_.append(np.random.choice(randomdummy_groups[o],replace=False,size=len(randomdummy_groups[o])))
    tiles_ = np.concatenate(tiles_)
    _, idx = np.unique(tiles_, return_index=True)
    return tiles_[np.sort(idx)]

In [ ]:
#create_tiles_list(order_)

In [ ]:
#weight = error_patch1.sum(axis=-1).reshape(-1) / error_patch1.sum()

nareas, d  = error_patch1.shape 

seed_dict = {'a':1,'b':2,'c':3,'d':4,'e':5}

#is_random_only = True
out_dict = {}
M_list = [10, 15, 30, 50, 5]
types_list  = ['active','random','randomdummy','activeopt']

for type_ in types_list:
    for M in M_list:
        for rand_option in seed_dict.keys():
            seed_ = seed_dict[rand_option]
            # print(M,seed_)

            uniform_samples = np.zeros(error_patch1.shape[0],dtype=np.int16)
            nonuniform_samples = np.zeros(error_patch1.shape[0],dtype=np.int16)

            np.random.seed(seed_)

            if type_ == 'random':
                selected_files = np.random.choice(tiles_avail,M, replace=False)
                key_ = f'{M}{rand_option}r'

            elif type_ == 'active':
                selected_files = np.random.choice(tiles_avail,M, replace=False, p=weights_sample)
                key_ = f'{M}{rand_option}'
            elif type_ == 'randomdummy':
                order_ = np.random.choice(keys_dummy,replace=False,size=6)
                tiles_shuffled = create_tiles_list(order_)
                selected_files = tiles_shuffled[:M+10]
                key_ = f'{M}{rand_option}rd'

            if type_ == 'activeopt':
                if rand_option == 'a':
                    # choose the top weights_ only for key a
                    selected_files = tiles_avail[np.argsort(weights_sample)[::-1]][:M]
                    key_ = f'{M}opt'
                    out_dict[key_] = [x.split('manual_annotations')[-1] for x in selected_files]
            else:
                out_dict[key_] = [x.split('manual_annotations')[-1] for x in selected_files]

            # print(f'Available {len(tiles_avail)}, selected {len(selected_files)}, Uncertainty {uniform_samples.sum()}, Quantization {nonuniform_samples.sum()}')


In [ ]:
out_dict.keys()

In [ ]:
sorted(out_dict['30a'])

In [ ]:
set_30a = [
        "/T48MWB/palm_group1_Bischel",
        "/T48MUA/palm_group1_Bischel",
        "/T47NND/palm_group3_Brunner",
        "/T47NMD/palm_group1_Bischel",
        "/T48MVB/palm_group1_Bischel",
        "/T48MUB/palm_group1_Brunner",
        "/T48MUB/palm_group3_Brunner",
        "/T47NMD/palm_group2_Bischel",
        "/T48MVB/palm_group3_Bischel",
        "/T47NQD/palm_group1",
        "/T46NHL/palm_group2_Stephane",
        "/T48NTK/palm_group2",
        "/T47MPV/palm_group1_Bischel",
        "/T48MVV/palm_group3_matthew",
        "/T47NNC/palm_group1_Brunner",
        "/T47NPC/palm_group2_jacopo",
        "/T47NLB/palm_group1_florian",
        "/T47NKF/palm_group2_bischel",
        "/T47NQE/palm_group1",
        "/T47NRF/palm_group1",
        "/T47NRA/palm_group2_Bischel",
        "/T47MQV/palm_group2",
        "/T47NLD/palm_group2_florian",
        "/T47NPA/palm_group1_Bischel",
        "/T47NQB/palm_group2",
        "/T47MRU/palm_group3_Bischel",
        "/T47MMU/palm_group3_julia",
        "/T48NUG/palm_group1",
        "/T48MUB/palm_group2_Brunner",
        "/T47NPC/palm_group3_jacopo"
    ]

In [ ]:
set_30a_pf = [
                "/T48MWB/palm_group1_Bischel",
        "/T48MUA/palm_group1_Bischel",
        "/T47NND/palm_group3_Brunner",
        "/T47NMD/palm_group1_Bischel",
        "/T48MVB/palm_group1_Bischel",
        "/T48MUB/palm_group1_Brunner",
        "/T48MUB/palm_group3_Brunner",
        "/T47NMD/palm_group2_Bischel",
        "/T48MVB/palm_group3_Bischel",
        "/T47NQD/palm_group1",
        "/T46NHL/palm_group2_Stephane",
        "/T48NTK/palm_group2",
        "/T47MPV/palm_group1_Bischel",
        "/T48MVV/palm_group3_matthew",
        "/T47NNC/palm_group1_Brunner",
        "/T47NPC/palm_group2_jacopo",
        "/T47NLB/palm_group1_florian",
        "/T47NKF/palm_group2_bischel",
        "/T47NQE/palm_group1",
        "/T47NRF/palm_group1",
        "/T47NRA/palm_group2_Bischel",
        "/T47MQV/palm_group2",
        "/T47NLD/palm_group2_florian",
        "/T47NPA/palm_group1_Bischel",
        "/T47NQB/palm_group2",
        "/T47MRU/palm_group3_Bischel",
        "/T47MMU/palm_group3_julia",
        "/T48NUG/palm_group1",
        "/T48MUB/palm_group2_Brunner",
        "/T47NPC/palm_group3_jacopo"
    ]

In [ ]:
set(set_30a_pf) == set(out_dict['30a'])

In [ ]:
out_dict['10opt']

In [ ]:
out_dict['15br']

In [ ]:
out_dict['15ar']

In [ ]:
print(len(out_dict['15ard']))
out_dict['15ard']

In [ ]:
print(len(out_dict['15ard']))
out_dict['15ard']

In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

In [ ]:
import json
save_path = '/home/pf/pfstaff/projects/andresro/barry_palm/data/2A/datasets'
filename = save_path+'/palm4748a_activesamples_aug29.json'
#with open(filename, 'w') as fp:
#    json.dump(out_dict, fp)

with open(filename, 'r') as fp:
   out_dict = json.load(fp)


In [ ]:
out_dict.keys()


In [ ]:
gdf = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(rois_middle[:,0], rois_middle[:,1]))

gdf_train = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(rois_middle[index_train][:,0], rois_middle[index_train][:,1]))

gdf_val = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(rois_middle[index_val][:,0], rois_middle[index_val][:,1]))

In [ ]:
out_dict.keys()

In [ ]:
out_dict['50opt']

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
countries = geopandas.read_file("/scratch/andresro/leon_work/barry_palm/data/labels/countries/3countries.shp")


plt.figure(figsize=(8, 10))

## Active
ax = plt.subplot(1,3, 1, projection=ccrs.PlateCarree())

index_bool = [x.split('manual_annotations')[-1] in out_dict['15opt'] for x in tiles_gt_all]

#plt.figure(figsize=(6,6))
# ax = plt.axes(projection=ccrs.PlateCarree())

world.plot(ax=ax,
    color='lightgray', edgecolor='white')

countries.plot(ax=ax,
    color='lightgray', edgecolor='white')

gdf.plot(ax=ax,legend=True,marker='.',
         edgecolor='black',
         facecolor='none',
         # color='green',
         label='Available'
        )

gdf.loc[index_train].plot(ax=ax, marker='.',
                               color='orange',
               label='Train'
        )

gdf.loc[index_val].plot(ax=ax, marker='.',
color='blue',
                label='Val'
         )


gdf[index_bool].plot(ax=ax, marker='.',
                                 color='red', 
                                 label='Active'
        )


minx, miny, maxx, maxy = gdf.total_bounds
margin_x = 2
margin_y = 2

ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)

ax.legend(loc='lower left')

## Naive
ax = plt.subplot(1,3,2, projection=ccrs.PlateCarree())


world.plot(ax=ax,
    color='lightgray', edgecolor='white')

countries.plot(ax=ax,
    color='lightgray', edgecolor='white')

gdf.plot(ax=ax,legend=True,marker='.',
         edgecolor='black',
         facecolor='none',
         # color='green',
         label='Available'
        )

gdf.loc[index_train].plot(ax=ax, marker='.',
                               color='orange',
               label='Train'
        )

gdf.loc[index_val].plot(ax=ax, marker='.',
color='blue',
                label='Val'
         )


gdf[index_bool].plot(ax=ax, marker='.',
                                 color='red', 
                                 label='Naïve'
        )

minx, miny, maxx, maxy = gdf.total_bounds
margin_x = 2
margin_y = 2

ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)





#gl = ax.gridlines(
#   crs=ccrs.PlateCarree(),
#    draw_labels=True) #x_inline=False, y_inline=False)
#gl.xformatter = LONGITUDE_FORMATTER
#gl.yformatter = LATITUDE_FORMATTER
#gl.xlabel_style = {'rotation':'vertical'}


# ax.legend(bbox_to_anchor=(1, 0.75), loc='upper left', ncol=1,
#          title="Sample Type")
ax.legend(loc='lower left')

# plt.title('Active Learning Selected Samples\n\n')
# plt.savefig(save_path_figs + 'active_learning_samples_15br.png', bbox_inches='tight', dpi= 300)


In [ ]:

#ref_key = '15opt'
#label_ = 'Active'

ref_key = '15br'
label_ = 'Manual'

#ref_key = '15brd'
#label_ = 'Naïve'

index_bool = [x.split('manual_annotations')[-1] in out_dict[ref_key] for x in tiles_gt_all]

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
countries = geopandas.read_file("/scratch/andresro/leon_work/barry_palm/data/labels/countries/3countries.shp")


plt.figure(figsize=(4,4))
ax = plt.axes(projection=ccrs.PlateCarree())


world.plot(ax=ax,
    color='lightgray', edgecolor='white')

countries.plot(ax=ax,
    color='lightgray', edgecolor='white')


gdf.plot(ax=ax,legend=True,marker='.',
         edgecolor='black',
         facecolor='none',
         # color='green',
         label='Available'
        )


gdf.loc[index_train].plot(ax=ax, marker='.',
                               color='orange',
               label='Train'
        )

gdf.loc[index_val].plot(ax=ax, marker='.',
color='blue',
                label='Val'
         )

gdf[index_bool].plot(ax=ax, marker='.',
                                 color='red', 
                                 label=label_
        )


minx, miny, maxx, maxy = gdf.total_bounds
margin_x = 2
margin_y = 2

ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)



gl = ax.gridlines(
crs=ccrs.PlateCarree(),
    draw_labels=False)
#gl = ax.gridlines(
#   crs=ccrs.PlateCarree(),
#    draw_labels=True) #x_inline=False, y_inline=False)
#gl.xformatter = LONGITUDE_FORMATTER
#gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'rotation':'vertical'}


# ax.legend(bbox_to_anchor=(1, 0.75), loc='upper left', ncol=1,
#          title="Sample Type")
ax.legend(loc='lower left')

# plt.title('Active Learning Selected Samples\n\n')
plt.savefig(f'{save_path_figs}active_learning_samples_{ref_key}.pdf', bbox_inches='tight', dpi= 300)


## Visualizing each patch available

In [ ]:
x_average = x_sum /  n[...,np.newaxis]

x_average1 = np.array([x_average[id_] for id_ in patch_id_gt])

In [ ]:
import numpy as np
from sklearn.manifold import TSNE


In [ ]:
x_average1.shape

In [ ]:
X = x_average.reshape(-1,320)
index_clean = ~np.any(np.isnan(X),axis=-1)
X = X[index_clean]

In [ ]:
model = TSNE(n_components=2).fit(X)

In [ ]:
X_embedded = model.transform(X)

In [ ]:
plt.scatter(X_embedded[:, 0], X_embedded[:, 1] ) #, c=X_embedded[:, 2] , cmap=plt.cm.Spectral)

In [ ]:
error_patch1 = np.array([error_patch[id_] for id_ in patch_id_gt])
std_patch1 = np.array([std_patches[id_] for id_ in patch_id_gt])



# Palm4

#### Reading GT

In [ ]:
# path_gt = '/scratch/andresro/leon_work/barry_palm/data/labels/manual_annotations'
path_gt = '/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/manual_annotations'

tiles_gt_all = glob.glob(path_gt+'/T*/palm*')

tiles_gt_all = sorted(tiles_gt_all)

In [ ]:
rois_gt_all = np.array([gp.get_positive_area_folder(tile_) for tile_ in tiles_gt_all])

def middle_roi(roi_list):
    return np.array([roi_list[:,0]+roi_list[:,2],roi_list[:,1]+roi_list[:,3]]).T/2

rois_middle = middle_roi(rois_gt_all)



In [ ]:
gdf = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(rois_middle[:,0], rois_middle[:,1]))


In [ ]:
# path_ = '/scratch/andresro/leon_work/barry_palm/data/labels/manual_annotations'
# index_  = [tiles_available.index(path_+x) for x in out_dict['10a']]
# index_bool = [x in index_ for x in list(gdf.index)]

In [ ]:
#gdf['Sample'] = 'No'
#gdf['Sample'].loc[nonuniform_samples > 0] = 'Non Uniform'
#gdf['Sample'].loc[uniform_samples > 0] = 'Random'

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
countries = geopandas.read_file("/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/countries/3countries.shp")

plt.figure(figsize=(10,10))
#fig, ax = plt.subplots(figsize=(5,5))

ax = plt.axes(projection=ccrs.PlateCarree())

#world.plot(ax=ax,
#    color='lightgray', edgecolor='white')

countries[countries.NAME_0 != 'Philippines'].plot(ax=ax,
     color='lightgray', edgecolor='white')


gdf.plot(ax=ax, marker='.')

minx, miny, maxx, maxy = countries[countries.NAME_0 != 'Philippines'].total_bounds
margin_x = 2
margin_y = 2


ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)

gl = ax.gridlines(
   crs=ccrs.PlateCarree(),
    draw_labels=True) #x_inline=False, y_inline=False)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

gl.xlabel_style = {'rotation':'vertical'}

# plt.title('Available samples')
# plt.savefig('active_learning_samples_10a.png')

#plt.savefig(save_path_figs+'3countries.png', 
##             bbox_extra_artists=(lgd,),
#            bbox_inches='tight')


## Choosing Base train and val samples

In [ ]:
def isin(string_,list_):
    return np.any([x in string_ for x in list_])

In [ ]:

# print(N)
# np.random.seed(1)
# index = np.arange(N)
# np.random.shuffle(index)

In [ ]:
valtiles_manual = 'T51NUB,T51NXB,T51NXA,T51MTT,T52MGE,T54MVS,T52MGD,T51MVU,T51MVR,T51MUR,T50MRD,T50MQD,T50MQC,T53MPT,T51MVS,T54MUC,T53MPR,T53MLV,T52MHD,T51NYB'
valtiles_manual = valtiles_manual.split(',')

In [ ]:
np.random.seed(1)

index_val_east = np.array([i for i,x in enumerate(tiles_gt_all) if isin(x, valtiles_manual)])

index_val_group2 = np.array([i for i,x in enumerate(tiles_gt_all) if 'group2' in x and not isin(x, valtiles_manual)])

np.random.shuffle(index_val_group2)

index_val_group2 = index_val_group2[:int(len(index_val_group2)*0.6)]

index_val_pre = np.concatenate((index_val_east,index_val_group2))


N = len(tiles_gt_all)

index1 = np.array([x for x in range(N) if not x in index_val_pre])

np.random.shuffle(index1)


index_train = index1 # [:-5]
#index_val = np.concatenate((index1[-5:],index_val_pre))
index_val = index_val_pre

print(f' total {N}, train {index_train.shape}, val {index_val.shape} (pre-selected {index_val_pre.shape})')

In [ ]:
tiles_train = np.array(tiles_gt_all)[index_train]
tiles_val = np.array(tiles_gt_all)[index_val]

assert len(set(tiles_train).intersection(set(tiles_val))) == 0


In [ ]:
len(tiles_val)

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
countries = geopandas.read_file("/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/countries/3countries.shp")


plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())

world.plot(ax=ax,
    color='lightgray', edgecolor='white')

countries.plot(ax=ax,
    color='lightgray', edgecolor='white')


#gdf.plot(ax=ax,legend=True, marker='.',
#          edgecolor='black',
#          facecolor='none',
         # color='green',
#         label='Available - Not Selected'
#        )


gdf.loc[index_train].plot(ax=ax, marker='.',
                                 color='orange',
               label='Train'
        )


gdf.loc[index_val].plot(ax=ax, marker='.',
                                color='blue',
                label='Val'
         )


minx, miny, maxx, maxy = gdf.total_bounds
margin_x = 2
margin_y = 2

ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)


gl = ax.gridlines(
   crs=ccrs.PlateCarree(),
    draw_labels=True) #x_inline=False, y_inline=False)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'rotation':'vertical'}

ax.legend(loc='upper right')

#ax.legend(bbox_to_anchor=(1, 0.75), loc='upper left', ncol=1,
#         title="Sample Type")
# plt.title('Initial dataset\n\n')
# plt.savefig('active_learning_samples_10a.png')
# plt.savefig(save_path_figs+'2_countries_base_dataset.png', bbox_inches='tight',dpi=300)


In [ ]:
#[x.split('manual_annotations')[-1] for x in tiles_train]

In [ ]:
out_dict = {'train':
           [x.split('manual_annotations')[-1] for x in tiles_train],
           'val':
           [x.split('manual_annotations')[-1] for x in tiles_val]}

In [ ]:
import json
save_path = '/home/pf/pfstaff/projects/andresro/barry_palm/data/2A/datasets'
filename = save_path+'/palm4_base.json'
#with open(filename, 'w') as fp:
#    json.dump(out_dict, fp)
with open(filename, 'rb') as fp:
    out_dict = json.load(fp)

### Reading ensembe predictions

In [ ]:
path = '/scratch/andresro/leon_work/sparse/inference/palm4_simpleA9_soft_ens5'

#path = '/scratch/andresro/leon_work/sparse/inference/palm4650_simpleA9_soft_mc5'

In [ ]:
tilenames = list({os.path.basename(x).split('_')[1] for x in glob.glob(path+'/R*')})
len(tilenames)

For every sample $i$ and dimension $j$:

$$ \sum_i ||X_i - Y ||^2_2 = \sum_i \big( \sum_j X_{i,j}^2 - 2 \sum_j (X_{i,j}*Y_j) +  \sum_j Y_j^2 \big)  $$

$$ \sum_j \big( \sum_i X_{i}^2 - 2 Y_j\sum_i X_{i} + I* Y_j^2 \big)  $$

In [ ]:
# Aggregate same locations
for tilename in tilenames:
    fname = f'{path}/{tilename}-5_preds_last.npz'
    if not os.path.isfile(fname):
        pred_dirs = glob.glob(f'{path}/*{tilename}/*.npz')
        if len(pred_dirs) > 0:

            # sum over several days in same locations
            x_sum1 = []
            x2_sum1 = []
            n1 = []

            for dir_ in pred_dirs:
                preds_last = np.load(dir_)

                x_sum2, x2_sum2, n2 = preds_last['x_sum'], preds_last['x2_sum'], preds_last['n']
                x_sum1.append(x_sum2)
                x2_sum1.append(x2_sum2)
                n1.append(n2)

            x_sum1 = np.stack(x_sum1,axis=0).sum(axis=0)
            x2_sum1 = np.stack(x2_sum1,axis=0).sum(axis=0)
            n1 = np.stack(n1,axis=0).sum(axis=0)

            np.savez(fname,x_sum=x_sum1,x2_sum=x2_sum1,
                            n=n1, lonlat=preds_last['lonlat'])
            print(fname,'saved')
        else:
            print(f'{path}/*{tilename}/*.npz has no files')


In [ ]:
compression_ = '12'

In [ ]:
x_sum = []
x2_sum = []
n = []
lonlat = []
std_patches=[]

is_reload = False
fname_all = f'{path}/all_statistics.npz'

if not os.path.isfile(fname_all) or is_reload:
    print('re-processing and saving')
    
    for tilename in tqdm(tilenames):

        fname = f'{path}/{tilename}-5_preds_last.npz'
        if os.path.isfile(fname):
            preds_last = np.load(fname)
            x_sum0, x2_sum0, n0, lonlat0 = preds_last['x_sum'], preds_last['x2_sum'], preds_last['n'], preds_last['lonlat']
            

            # Adding nan values to patch statistics
            down100_file = f'{path}/{tilename}_5_preds_reg_{compression_}_{compression_}_down100.vrt'
            ds = gdal.Open(down100_file)
            down_array = ds.ReadAsArray()
            is_nan_array = interpPatches((down_array==99), ref_shape=(100,100)).squeeze() > 0.5
            is_nan_array = is_nan_array.reshape(-1)

            x_sum0[is_nan_array] = np.nan
            x2_sum0[is_nan_array] = np.nan
            n0[is_nan_array] = 0
            
            x_sum.append(x_sum0)
            x2_sum.append(x2_sum0)
            n.append(n0)
            lonlat.append(lonlat0)

            # Adding mean uncertainty per patch
            std_file = f'{path}/std/{tilename}_5_preds_reg_{compression_}_std_{compression_}.tif'
            ds = gdal.Open(std_file)
            std_array = ds.ReadAsArray()
            std_array[std_array == 99] = np.nan
            std_patch = block_reduce(std_array,(110,110),np.mean)
            std_patches.append(std_patch.reshape(-1))

    x_sum = np.stack(x_sum,axis=0)
    x2_sum = np.stack(x2_sum,axis=0)
    n = np.stack(n,axis=0)
    lonlat = np.stack(lonlat,axis=0)
    std_patches = np.stack(std_patches,axis=0)
    
    np.savez(fname_all,x_sum=x_sum,
            x2_sum=x2_sum,
            n=n, lonlat=lonlat,
            std_patches = std_patches)    
else:
    print('reading',fname_all)
    preds_all = np.load(fname_all)

    x_sum = preds_all['x_sum']
    x2_sum = preds_all['x2_sum']
    n = preds_all['n']
    lonlat = preds_all['lonlat']
    std_patches = preds_all['std_patches']

In [ ]:
#is_nan_patch = np.isnan(x_sum[...,0])
# is_nan_patch = n == 0
is_nan_patch = np.logical_or(n == 0,np.isnan(std_patches))

In [ ]:
(n == 0).mean(), is_nan_patch.mean()

In [ ]:
is_nan_patch.mean()

In [ ]:
is_nan_patch.shape,std_patches.shape

In [ ]:
plt.scatter(lonlat[is_nan_patch][...,0], lonlat[is_nan_patch][...,1],
            #c=np.log(weight_sorted[:top_k]),
            # c=weight_sorted[:top_k],
            cmap='magma_r', marker='.')

In [ ]:
plt.scatter(lonlat[~is_nan_patch][...,0], lonlat[~is_nan_patch][...,1],
            #c=np.log(weight_sorted[:top_k]),
            # c=weight_sorted[:top_k],
            cmap='magma_r', marker='.')

In [ ]:
x_sum[~is_nan_patch].shape

In [ ]:
mean_global = np.sum(x_sum[~is_nan_patch], axis=0) / n.sum()

# error means quantization error not std. dev.
error_patch = (x2_sum[~is_nan_patch] - 2*x_sum[~is_nan_patch]*mean_global + n.sum() * (mean_global**2))

# error_patch = np.clip(error_patch,0,np.percentile(error_patch,99))

error_global = np.sum(error_patch, axis=0)

In [ ]:
mean_global.shape, error_patch.shape

In [ ]:
#mean_global = np.nansum(x_sum, axis=(0,1)) / n.sum()

# error means quantization error not std. dev.
#error_patch = (x2_sum - 2*x_sum*mean_global + n.sum() * (mean_global**2))
#error_global = np.nansum(error_patch, axis=(0,1))

In [ ]:
p_min = 0
p_max = 99

uncertainty = np.nan_to_num(std_patches[~is_nan_patch][...,np.newaxis])

weight_div = error_patch.sum(axis=-1).reshape(-1) / np.sum(error_patch)

weight_div = np.clip(weight_div,np.percentile(weight_div,p_min),np.percentile(weight_div,p_max))
weight_div = weight_div / weight_div.sum()

weight_unc = (uncertainty).sum(axis=-1).reshape(-1) / np.sum(uncertainty)

#weight_unc = np.clip(weight_unc,np.percentile(weight_unc,p_min),np.percentile(weight_unc,p_max))
#weight_unc = weight_unc / weight_unc.sum()


weight_score = (weight_div + weight_unc)/2.0
weight_score = weight_score/weight_score.sum()

In [ ]:
max_perc = 99

f, ax = plt.subplots(2,2, figsize=(10,5))
ax[0,0].plot(weight_div,'x')
ax[0,1].plot(weight_unc,'x')

_ = ax[1,0].hist(np.clip(weight_div,0,np.percentile(weight_div,max_perc)))
_ = ax[1,1].hist(np.clip(weight_unc,0,np.percentile(weight_unc,max_perc)))

In [ ]:
np.percentile(weight_div,90), np.percentile(weight_div,99), np.percentile(weight_div,100)

In [ ]:
#index = np.random.choice(weight.shape[0],100000)
p_min = 0
p_max = 100

index_inlier = np.logical_and.reduce(
                      (weight_div > np.percentile(weight_div,p_min),
                       weight_div < np.percentile(weight_div,p_max),
                       weight_unc > np.percentile(weight_unc,p_min),
                       weight_unc < np.percentile(weight_unc,p_max),
                      ))


g = sns.jointplot(x=weight_div[index_inlier], y=weight_unc[index_inlier], kind="hex",
                #  xlim=(np.percentile(weight,p_min),np.percentile(weight,p_max)),
                # ylim=(np.percentile(weight1,p_min),np.percentile(weight1,100))) #, data=tips))
                 )

g.ax_joint.set_xlabel('Quant Error')
g.ax_joint.set_ylabel('Uncertainty')
#g.ax_joint.set_xlim(0,1)


# g.ax_joint.set_xlim(np.percentile(weight_div,p_min),np.percentile(weight_div,80))
#g.ax_joint.set_ylim(np.percentile(weight1,p_min),np.percentile(weight1,100))
# g.ax_joint.set_xticks([])
# g.ax_joint.set_yticks([])



# plt.savefig(save_path_figs + 'uncertainty_vs_quant_all.png', bbox_inches='tight', dpi= 300)

In [ ]:
ids_sorted = np.argsort(weight_score)[::-1] 

weight_sorted = weight_score[ids_sorted]
lonlat_sorted = lonlat[~is_nan_patch][ids_sorted]

In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

In [ ]:
index_train = [x.split('manual_annotations')[-1] in out_dict['train'] for x in tiles_gt_all]
index_val = [x.split('manual_annotations')[-1] in out_dict['val'] for x in tiles_gt_all]

# index_bool = [x.split('manual_annotations')[-1] in out_dict['30opt'] for x in tiles_gt_all]


In [ ]:
N_active = 30

gdf = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(rois_middle[:,0], rois_middle[:,1]))

gdf = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(rois_middle[:,0], rois_middle[:,1]))


gdf_active = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(lonlat_sorted[:N_active,0], lonlat_sorted[:N_active,1]))


In [ ]:
top_k = 1000
plt.scatter(lonlat_sorted[:top_k,0], lonlat_sorted[:top_k,1],
            #c=np.log(weight_sorted[:top_k]),
            c=weight_sorted[:top_k],
            cmap='magma_r', marker='.')

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
countries = geopandas.read_file("/scratch/andresro/leon_work/barry_palm/data/labels/countries/3countries.shp")


plt.figure(figsize=(6,6))
ax = plt.axes(projection=ccrs.PlateCarree())


world.plot(ax=ax,
    color='lightgray', edgecolor='white')

#countries.plot(ax=ax,
#    color='lightgray', edgecolor='white')

gdf.loc[index_train].plot(ax=ax, marker='.',
                               color='orange',
               label='Train'
        )

gdf.loc[index_val].plot(ax=ax, marker='.',
color='blue',
                label='Val'
         )

gdf_active.plot(ax=ax, marker='.',
                                 color='red', 
                                 label='Active'
        )


minx, miny, maxx, maxy = gdf.total_bounds
margin_x = 2
margin_y = 2


minx1, miny1, maxx1, maxy1 = gdf_active.total_bounds

minx = min(minx,minx1)
miny = min(miny,miny1)
maxx = max(maxx,maxx1)
maxy = max(maxy,maxy1)


ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)




gl = ax.gridlines(
   crs=ccrs.PlateCarree(),
    draw_labels=True) #x_inline=False, y_inline=False)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'rotation':'vertical'}


# ax.legend(bbox_to_anchor=(1, 0.75), loc='upper left', ncol=1,
#          title="Sample Type")
#ax.legend() # loc='lower left')

# plt.title('Active Learning Selected Samples\n\n')
# plt.savefig(save_path_figs + 'active_learning_samples_10opt.', bbox_inches='tight', dpi= 300)


In [ ]:
import simplekml

kml = simplekml.Kml()
for key, val in enumerate(lonlat_sorted[:N_active]):
    #lat, lon = gp.to_latlon(val[1],val[0],ds_pred)
    lon, lat = val
    pnt = kml.newpoint(name=str(key), coords=[(lon, lat)])
    
# kml.save(f'/scratch2/Dropbox/Dropbox/0_phd/temp/{N_active}_New_samples_palm4.kml')

## Using core-set for a k-mean solution

#### Sample C samples to create a core-set

In [ ]:
(n[~is_nan_patch] == 0).sum()

In [ ]:
is_nan_patch.shape, weight_score.shape

In [ ]:
n_bins = 4

bins_long = np.linspace(np.min(lonlat[~is_nan_patch][:,0]),np.max(lonlat[~is_nan_patch][:,0]),n_bins+1)

In [ ]:
bins_index = []

for bin_ in range(n_bins):
    bins_index.append(np.logical_and(bins_long[bin_]<=lonlat[~is_nan_patch][:,0], bins_long[bin_+1]>=lonlat[~is_nan_patch][:,0]))
    
bins_index = np.array(bins_index)

In [ ]:
C = 100000
N_samples = weight_score.shape[0]

# w_ = weight_score[n[~is_nan_patch] > 0]/weight_score[n[~is_nan_patch] > 0].sum()
# w_ = weight_score[n.reshape(-1) > 0]/weight_score[n.reshape(-1) > 0].sum()
# w_ = weight_score[np.logical_and(n.reshape(-1) > 0,np.percentile(weight_score,99))]/weight_score[np.logical_and(n.reshape(-1) > 0,np.percentile(weight_score,99))].sum()
# w_ = weight_unc[n.reshape(-1) > 0]/weight_unc[n.reshape(-1) > 0].sum()

samples_id = np.arange(N_samples) # [n[~is_nan_patch] > 0]

np.random.seed(1)

#core_set = np.random.choice(samples_id,size=C, replace=True, p=weight_score)

# ids_sorted = np.argsort(weight_score)[::-1] 
# core_set = samples_id[ids_sorted][:C]

## top-k with bins

N_bins = (C *bins_index.sum(axis=1) / bins_index.sum()).astype(np.int)


core_set = []
for bin_ in range(n_bins):
    index_ = bins_index[bin_]
    ids_sorted = np.argsort(weight_score[index_])[::-1]
    samples_id_sorted = samples_id[index_][ids_sorted]
    n_ = N_bins[bin_]        
    core_set.append(samples_id_sorted[:C])

core_set = np.stack(core_set)

## top-k no bins
# ids_sorted = np.argsort(weight_score)[::-1] 
# core_set = samples_id[ids_sorted][:C]

# ids_sorted = np.argsort(weight_unc)[::-1] 
# core_set = samples_id[ids_sorted][:C//2]
# ids_sorted = np.argsort(weight_div)[::-1] 
# core_set = np.concatenate((core_set,samples_id[ids_sorted][:C//2]))

core_set_weight = 1 / (C* weight_score[core_set.reshape(-1)])
# core_set_weight = 1 / (C * weight_unc[core_set])

# key_ = f'{M}{rand_option}'

In [ ]:
plt.scatter(lonlat[~is_nan_patch][:,0], lonlat[~is_nan_patch][:,1],
            #c=np.log(weight_sorted[:top_k]),
            c=weight_score,
            cmap='magma_r', marker='.')

In [ ]:
i_ = 1

plt.scatter(lonlat[~is_nan_patch][bins_index[i_]][:,0],
            lonlat[~is_nan_patch][bins_index[i_]][:,1],
            #c=np.log(weight_sorted[:top_k]),
            c=weight_score[bins_index[i_]],
            cmap='magma_r', marker='.')

In [ ]:
i_ = 3

plt.scatter(lonlat[~is_nan_patch][core_set[i_]][:,0],
            lonlat[~is_nan_patch][core_set[i_]][:,1],
            #c=np.log(weight_sorted[:top_k]),
            c=weight_score[core_set[i_]],
            cmap='magma_r', marker='.')

In [ ]:
g = sns.jointplot(x=weight_div[core_set], y=weight_unc[core_set], kind="hex")

g.ax_joint.set_xlabel('Quant Error')
g.ax_joint.set_ylabel('Uncertainty')
# g.ax_joint.set_xlim(np.percentile(weight_div,p_min),np.percentile(weight_div,80))
#g.ax_joint.set_ylim(np.percentile(weight1,p_min),np.percentile(weight1,100))
# g.ax_joint.set_xticks([])
# g.ax_joint.set_yticks([])


# plt.savefig(save_path_figs + 'uncertainty_vs_quant_all.png', bbox_inches='tight', dpi= 300)

In [ ]:
coreset_lonlat_coords = lonlat[~is_nan_patch][core_set]

x_mean_coreset = x_sum[~is_nan_patch][core_set] / (n[~is_nan_patch][core_set][...,np.newaxis])

In [ ]:
x_mean_coreset.shape

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
n_clusters = 50

N_bins = (n_clusters *bins_index.sum(axis=1) / bins_index.sum()).astype(np.int)
center_pixels_index = []
for i in range(n_bins):
    model = KMeans(n_clusters=N_bins[i], random_state=0)
#     model.fit(x_mean_coreset[i*C:(i+1)*C], sample_weight=core_set_weight[i*C:(i+1)*C])
    model.fit(x_mean_coreset[i], sample_weight=core_set_weight[i])

    index_center = np.argmin(model.transform(x_mean_coreset[i]), axis=0) # + i*C
    
    center_pixels_index.append(index_center)

# center_pixels_index = np.concatenate(center_pixels_index)

In [ ]:
centers_index = []
for i in range(n_bins):
    centers_index.append(center_pixels_index[i]+i*C)

In [ ]:
# core_set = core_set.reshape(-1)

In [ ]:
# n_clusters = 50

# model = KMeans(n_clusters=n_clusters, random_state=0)
# model.fit(x_mean_coreset, sample_weight=core_set_weight)

# cluster_id = model.predict(x_mean_coreset)

# center_pixels_index = np.argmin(model.transform(x_mean_coreset), axis=0)
# center_lonlat_coords = coreset_lonlat_coords[center_pixels_index]

# # center_lonlat_coords = center_lonlat_coords[np.argsort(center_lonlat_coords[:,0],axis=0)]

In [ ]:
g = sns.jointplot(x=weight_div[core_set], y=weight_unc[core_set], kind="hex")


g.ax_joint.set_xlabel('Quant Error')
g.ax_joint.set_ylabel('Uncertainty')
# g.ax_joint.set_xlim(np.percentile(weight_div,p_min),np.percentile(weight_div,80))
#g.ax_joint.set_ylim(np.percentile(weight1,p_min),np.percentile(weight1,100))
# g.ax_joint.set_xticks([])
# g.ax_joint.set_yticks([])

for i in range(n_bins):
    g.ax_joint.scatter(x=weight_div[core_set[i]][center_pixels_index[i]],
                    y=weight_unc[core_set[i]][center_pixels_index[i]], c='red')


plt.savefig(save_path_figs + 'uncertainty_vs_quant_coreset_sept30.png', bbox_inches='tight', dpi= 300)

In [ ]:
from sklearn.manifold import TSNE

Y = TSNE(n_components=2, n_jobs=-1).fit_transform(x_mean_coreset.reshape(-1,320))

In [ ]:
g = sns.jointplot(Y[...,0],Y[...,1],kind='kde')

# sns.jointplot(Y[patch_id_gt][:,0],Y[patch_id_gt][:,1])
Y_ = Y.reshape(n_bins,-1,2)
for i in range(n_bins):
    g.ax_joint.scatter(x=Y_[i][center_pixels_index[i]][...,0],
                       y=Y_[i][center_pixels_index[i]][...,1],
                       c='red')

# g.ax_joint.scatter(Y[center_pixels_index][:,0],Y[center_pixels_index][:,1], c='red')
# g.ax_joint.scatter(Y[patch_id_gt][center_pixels_index][:,0],Y[patch_id_gt][center_pixels_index][:,1], c='white', marker='.', facecolor=None)


plt.savefig(save_path_figs + 'tsne_palm4_ALset.pdf', bbox_inches='tight', dpi= 300)

In [ ]:
center_lonlat_coords = []
for i in range(n_bins):
    center_lonlat_coords.append(coreset_lonlat_coords[i][center_pixels_index[i]])
center_lonlat_coords = np.concatenate(center_lonlat_coords)

In [ ]:
gdf_coreset = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(coreset_lonlat_coords.reshape(-1,2)[:,0], coreset_lonlat_coords.reshape(-1,2)[:,1]))

gdf_active = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(center_lonlat_coords.reshape(-1,2)[:,0], center_lonlat_coords.reshape(-1,2)[:,1]))



In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
#countries = geopandas.read_file("/scratch/andresro/leon_work/barry_palm/data/labels/countries/3countries.shp")


plt.figure(figsize=(6,6))
ax = plt.axes(projection=ccrs.PlateCarree())


# gdf_coreset.plot(ax=ax, marker='o',
#                                  color='lightgray', 
#                                  label='Coreset'
#         )


world.plot(ax=ax,
    color='lightgray', edgecolor='white')

#countries.plot(ax=ax,
#    color='lightgray', edgecolor='white')




gdf.loc[index_train].plot(ax=ax, marker='.',
                               color='orange',
               label='Train'
        )

gdf.loc[index_val].plot(ax=ax, marker='.',
color='blue',
                label='Val'
         )

gdf_active.plot(ax=ax, marker='.',
                                 color='red', 
                                 label='Active'
        )


minx, miny, maxx, maxy = gdf.total_bounds
margin_x = 2
margin_y = 2


minx1, miny1, maxx1, maxy1 = gdf_coreset.total_bounds

minx = min(minx,minx1)
miny = min(miny,miny1)
maxx = max(maxx,maxx1)
maxy = max(maxy,maxy1)


ax.set_xlim(minx-margin_x, maxx+margin_x)
ax.set_ylim(miny-margin_y, maxy+margin_y)


gl = ax.gridlines(
   crs=ccrs.PlateCarree(),
    draw_labels=True) #x_inline=False, y_inline=False)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'rotation':'vertical'}


# ax.legend(bbox_to_anchor=(1, 0.75), loc='upper left', ncol=1,
#          title="Sample Type")
ax.legend() # loc='lower left')

# plt.title('Active Learning Selected Samples\n\n')
# 

plt.savefig(save_path_figs + 'palm4_30_ALsamples_sept25.pdf', bbox_inches='tight', dpi= 300)


In [ ]:
import simplekml

kml = simplekml.Kml()
for key, val in enumerate(center_lonlat_coords):
    #lat, lon = gp.to_latlon(val[1],val[0],ds_pred)
    lon, lat = val
    pnt = kml.newpoint(name=str(key), coords=[(lon, lat)])
    
kml.save(f'/scratch2/Dropbox/Dropbox/0_phd/temp/{n_clusters}samples_coreset100k_combinedscore.kml')

## Uncertainty only

In [ ]:

#path = os.path.dirname(data_dir)
pred_tifs = glob.glob(f'{path}/{tilename}*.tif')
std_tifs = glob.glob(f'{path}/std/{tilename}*std*.tif')


In [ ]:
ds = gdal.Open(std_tifs[0])

patch_size = 150
border = 20
patch1 = 150  - border*2
print(patch1)
ds.RasterXSize/patch1, ds.RasterYSize /patch1

In [ ]:
ds = gdal.Open(std_tifs[0])
preds_array = ds.ReadAsArray()

In [ ]:
preds_array = ds.ReadAsArray()

preds_array[preds_array == 99] = np.nan
preds_array1 = block_reduce(preds_array,(110,110),np.mean)
plt.imshow(preds_array)

In [ ]:


preds_array1 = block_reduce(preds_array,(110,110),np.mean)

In [ ]:
plt.imshow(preds_array1)

In [ ]:
x = preds_last['lonlat'][:,0].reshape(100,100)
y = preds_last['lonlat'][:,1].reshape(100,100)
z = error_patch.mean(axis=1).reshape(100,100)
#X, Y = np.meshgrid(x, y)

plt.pcolormesh(x, y,z) # shading='auto') #, cmap='Reds')

In [ ]:
ds = gdal.Open(path_)
ds

In [ ]:
#base_path = '/scratch/andresro/leon_work/sparse/inference/'
base_path = '/home/pf/pfstaff/projects/andresro/sparse/inference_leon/'
path_ = base_path + "borneo_simpleA9_mc10/0_EPSG_32749_down400.tif"
ds_pred = gdal.Open(path_)
array_preds = ds_pred.ReadAsArray()
ref_proj = ds_pred.GetGeoTransform()

path_ = base_path +"borneo_simpleA9_mc10/std/0_EPSG_32749_down400.tif"
ds = gdal.Open(path_)
warp_opts = gdal.WarpOptions(
    format="VRT", 
    xRes=ref_proj[1],
    yRes=ref_proj[5],
    dstNodata='nan')
gdal.Warp('/vsimem/std_reprojected.vrt', ds, options=warp_opts)
ds = gdal.Open('/vsimem/std_reprojected.vrt')
array_std = ds.ReadAsArray()



In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(121)
plt.title('Predicted density')
plt.imshow(np.clip(array_preds,0.4,1))
plt.axis('off')

plt.subplot(122)
plt.title('Model Uncertainty')
plt.imshow(np.clip(array_std,0.3,0.8), cmap='Reds')
plt.axis('off')

plt.savefig('/scratch2/Dropbox/Dropbox/0_phd/temp/density_uncertainty_borneo.png',bbox_inches='tight', dpi=200)

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(121)

plt.title('Model Uncertainty')
plt.imshow(np.clip(array_std,0.3,0.8), cmap='Reds')
plt.axis('off')

plt.subplot(122)

array1 = np.logical_and(array_std >0.4, array_preds > 0.15)*1.0
array1[np.isnan(array_std)] = np.nan
plt.title('High uncertainty and medium or higher density')
plt.imshow(array1, cmap = 'magma')
plt.axis('off')

plt.savefig('/scratch2/Dropbox/Dropbox/0_phd/temp/density_uncertainty_borneo_1.png',bbox_inches='tight', dpi=200)

In [ ]:
pixel_coords = np.argwhere(np.logical_and(array_std > 0.4, array_preds > 0.15))
latlon_coords = np.array([gp.to_latlon(y,x,ds_pred) for x, y in pixel_coords])

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
n_clusters = 15

model = KMeans(n_clusters=n_clusters, random_state=0)
model.fit(pixel_coords)

In [ ]:
cluster_id = model.predict(pixel_coords)

In [ ]:
center_pixels_index = np.argmin(model.transform(pixel_coords), axis=0)

center_latlon_coords = latlon_coords[center_pixels_index]

center_latlon_coords = center_latlon_coords[np.argsort(center_latlon_coords[:,0],axis=0)]

In [ ]:
#plt.scatter(-pixel_coords[:,0], pixel_coords[:,1], c=cluster_id, cmap='tab20',marker='.')
#plt.scatter(-center_pixel_coords[:,0], center_pixel_coords[:,1], marker='x')

plt.figure(figsize=(10,5))

plt.subplot(121)

array1 = np.logical_and(array_std >0.4, array_preds > 0.15)*1.0
array1[np.isnan(array_std)] = np.nan
plt.title('High uncertainty and medium or higher density')
plt.imshow(array1, cmap = 'magma')
plt.axis('off')

plt.subplot(122)

plt.title('Cluster areas')

plt.scatter(latlon_coords[:,1], latlon_coords[:,0], c=cluster_id, cmap='tab20',marker='.')
plt.scatter(center_latlon_coords[:,1], center_latlon_coords[:,0], marker='x')
plt.axis('off')

plt.savefig('/scratch2/Dropbox/Dropbox/0_phd/temp/density_uncertainty_borneo_centers.png',bbox_inches='tight', dpi=200)

In [ ]:
center_latlon_coords.shape

In [ ]:
import simplekml

kml = simplekml.Kml()
for key, val in enumerate(center_latlon_coords):
    #lat, lon = gp.to_latlon(val[1],val[0],ds_pred)
    lat, lon = val
    pnt = kml.newpoint(name=str(key), coords=[(lon, lat)])
    
#kml.save(f'/scratch2/Dropbox/Dropbox/0_phd/temp/{n_clusters}_New_samples.kml')

In [ ]:
!echo $PF

In [ ]:
import seaborn as sns
import pandas as pd

#no_na = np.logical_and(~np.isnan(array_preds.reshape(-1)),~np.isnan(array_std.reshape(-1)))
df = pd.DataFrame({"pred": array_preds.reshape(-1), "std": array_std.reshape(-1)})
df.dropna(inplace=True)
#
#df = df[np.logical_and(df['pred'] > 0.2,df['std'] > 0.0)]
print(df.shape)

In [ ]:
hexplot = sns.jointplot("pred", "std", data = df ,kind="hex", space=1,bins='log') #, xlim=(0,0.5), ylim=(0,0.6))
#hexplot.fig.axhline(y=0.5, color='r', linestyle='-')

plt.subplots_adjust(left=0.2, right=0.8, top=0.8, bottom=0.2)  # shrink fig so cbar is visible
# make new ax object for the cbar
cbar_ax = hexplot.fig.add_axes([.85, .25, .05, .4])  # x, y, width, height
plt.colorbar(cax=cbar_ax)

hexplot.ax_joint.plot([0, 1.25], [0.4, 0.4], linewidth=2, c='r')
hexplot.ax_joint.plot([0.15, 0.15], [0, 0.55], linewidth=2, c='g')

plt.show()



In [ ]:
H.shape, X.shape,Y.shape

In [ ]:
n_bins = 100
xedges = np.linspace(0,1.2,n_bins)
yedges = np.linspace(0,0.5,n_bins)

#xedges = np.logspace(-.0,1.2,500)
#yedges = np.logspace(0,0.5,500)

H, xedges, yedges = np.histogram2d(df['pred'], df['std'], bins=(xedges, yedges))
H = H.T
fig = plt.figure(figsize=(10, 5))


ax = fig.add_subplot(111,
        aspect='equal')
X, Y = np.meshgrid(xedges, yedges)
im = ax.pcolormesh(X, Y, np.log(H+1e-7), cmap='Blues')
#ax.pcolormesh(X, Y, H, cmap='Blues')

ax = plt.subplots_adjust(left=0.2, right=0.8, top=0.8, bottom=0.2)  # shrink fig so cbar is visible
# make new ax object for the cbar
plt.colorbar(im, ax)




In [ ]:
hexplot = sns.jointplot("pred", "std", data = df ,kind="hex", space=1,
                        bins='log',
                        xlim=(0.15,1.25), ylim=(0.4,0.6))

plt.subplots_adjust(left=0.2, right=0.8, top=0.8, bottom=0.2)  # shrink fig so cbar is visible
# make new ax object for the cbar
cbar_ax = hexplot.fig.add_axes([.85, .25, .05, .4])  # x, y, width, height
plt.colorbar(cax=cbar_ax)
plt.show()


In [ ]:
sns.jointplot("pred", "std", data = df,kind="hex",ratio=3, xlim=(-.1,0.2), ylim=(0.1,0.3),bins='log')

In [ ]:
sns.jointplot("pred", "std", data = df[np.logical_and(df['pred'] > 0.15,df['std'] > 0.4)] ,bins='log',kind="hex") #, xlim=(0,0.5), ylim=(0,0.6))

In [ ]:
np.array(df[np.logical_and(df['pred'] > 0.2,df['std'] > 0.2)])

In [ ]:
from sklearn.manifold import TSNE

#X_embedded = TSNE(n_components=2).fit_transform(np.array(df[np.logical_and(df['pred'] > 0.2,df['std'] > 0.2)]))
X_embedded = TSNE(n_components=2).fit_transform(np.array(df))



In [ ]:
plt.scatter(X_embedded[:,0], X_embedded[:,1])

In [ ]:
tile_ = 'T49MET'
path_ = f"/scratch/andresro/leon_work/sparse/inference/borneo_simpleA9_mc10/{tile_}_10_preds_reg_0_0_down20_EPSG_32749.vrt"
ds = gdal.Open(path_)
array_preds = ds.ReadAsArray()

path_ = f"/scratch/andresro/leon_work/sparse/inference/borneo_simpleA9_mc10/std/{tile_}_10_preds_reg_0_std_0_down20_EPSG_32749.vrt"

ds = gdal.Open(path_)
array_std = ds.ReadAsArray()



In [ ]:
plt.imshow(np.clip(array_preds,0.4,1))

In [ ]:
plt.imshow(np.clip(array_std,0.3,0.8))

In [ ]:
import seaborn as sns
import pandas as pd

#no_na = np.logical_and(~np.isnan(array_preds.reshape(-1)),~np.isnan(array_std.reshape(-1)))
df = pd.DataFrame({"pred": array_preds.reshape(-1), "std": array_std.reshape(-1)})
df.dropna(inplace=True)
#

In [ ]:
sns.jointplot("pred", "std", data = df ,kind="kde") , xlim=(-0.25,1.5), ylim=(0,0.6))